Deep Learning Models -- A collection of various deep learning architectures, models, and tips for TensorFlow and PyTorch in Jupyter Notebooks.
- Author: Sebastian Raschka
- GitHub Repository: https://github.com/rasbt/deeplearning-models

In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch

Author: Sebastian Raschka

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 8.30.0

torch: 2.6.0+cu126



# Replacing Fully-Connnected by Equivalent Convolutional Layers

In [2]:
import torch

Assume we have a 2x2 input image:  
假设我们有一个2x2的输入图像：

In [3]:
inputs = torch.tensor([[[[1., 2.],
                         [3., 4.]]]])

inputs.shape

torch.Size([1, 1, 2, 2])

## Fully Connected

A fully connected layer, which maps the 4 input features to 2 outputs, would be computed as follows:  
一个全连接层，将4个输入特征映射到2个输出，将按如下方式计算：



In [4]:
fc = torch.nn.Linear(4, 2)

weights = torch.tensor([[1.1, 1.2, 1.3, 1.4],
                        [1.5, 1.6, 1.7, 1.8]])
bias = torch.tensor([1.9, 2.0])
fc.weight.data = weights
fc.bias.data = bias

In [5]:
torch.relu(fc(inputs.view(-1, 4)))

tensor([[14.9000, 19.0000]], grad_fn=<ReluBackward0>)

## Convolution with Kernels equal to the input size

![](../images/fc-to-conv/fc-to-conv-1.png)

We can obtain the same outputs if we use convolutional layers where the kernel size is the same size as the input feature array:  
如果我们使用卷积层，并且卷积核的大小与输入特征数组相同，我们可以获得相同的输出：

In [6]:
conv = torch.nn.Conv2d(in_channels=1,
                       out_channels=2,
                       kernel_size=inputs.squeeze(dim=(0)).squeeze(dim=(0)).size())
print(conv.weight.size())
print(conv.bias.size())

torch.Size([2, 1, 2, 2])
torch.Size([2])


In [7]:
conv.weight.data = weights.view(2, 1, 2, 2)
conv.bias.data = bias

In [8]:
torch.relu(conv(inputs))

tensor([[[[14.9000]],

         [[19.0000]]]], grad_fn=<ReluBackward0>)

## Convolution with 1x1 Kernels

![](../images/fc-to-conv/fc-to-conv-2.png)

Similarly, we can replace the fully connected layer using a convolutional layer when we reshape the input image into a num_inputs x 1 x 1 image:  
类似地，当我们将输入图像重塑为num_inputs x 1 x 1图像时，我们可以使用卷积层替代全连接层：

In [9]:
conv = torch.nn.Conv2d(in_channels=4,
                       out_channels=2,
                       kernel_size=(1, 1))

conv.weight.data = weights.view(2, 4, 1, 1)
conv.bias.data = bias
torch.relu(conv(inputs.view(1, 4, 1, 1)))

tensor([[[[14.9000]],

         [[19.0000]]]], grad_fn=<ReluBackward0>)